序：
本说明包括从处理数据到输出eval结果的全过程

1、process data：
首先从原路径中读取corpus文件：/share/peitian/Data/Datasets/llm-embedder/qa/msmarco/corpus.json
之后调用process_corpus.py：
python scripts/process_corpus.py --collection-path /share/peitian/Data/Datasets/llm-embedder/qa/msmarco/corpus.json --output-folder /share/yutao/yifei/bm25_data/corpus

需要提前在主文件旁新建一个文件夹叫“bm25_data”
它会在主目录旁边新创建一个corpus文件夹，里面有9个.json文件

之后从原路径中读取queries文件：/share/peitian/Data/Datasets/llm-embedder/qa/msmarco/train.json

之后调用process_queries.py：
python scripts/process_queries.py --collection-path /share/peitian/Data/Datasets/llm-embedder/qa/msmarco/train.json --output-folder /share/yutao/yifei/bm25_data/queries

它会在主目录旁边新创建一个queries文件夹，里面有41个.tsv文件

这之后，corpus的json文件中每一条数据格式为：{"id": id, "contents": contents}
queries的json文件中每一条数据格式为：{id query}

2、indexing
在bm25_data里创建一个文件名为index_msmarco，用于存储index
之后调用命令：
target/appassembler/bin/IndexCollection \
  -collection JsonCollection \
  -input /share/yutao/yifei/bm25_data/corpus \
  -index /share/yutao/yifei/bm25_data/index_msmarco \
  -generator DefaultLuceneDocumentGenerator \
  -threads 9 -storePositions -storeDocvectors -storeRaw 

3、retrieval
调用命令：
python scripts/mk_command.py

它会对之前保存的41个query的.tsv文件分别创建retrieval的命令，将所有命令保存到“commands.txt”中

之后调用命令：bash commands.txt

在bm25_data中构造了41个retrieval结果，分别保存为f"run.msmarco-passage.{num}.tsv"

之后合并结果：

调用命令
python BM25-evaluation/scripts/summary_runs.py --input-folder /share/yutao/yifei/bm25_data/output_runs --output-file /share/yutao/yifei/bm25_data/output_runs/run.msmarco.tsv

它会生成最终的结果文件：bm25_data/output_runs/run.msmarco.tsv
文件中每一行格式为：qid docid   rank，中间用'\t'隔开

之后bm25_data/output_runs中其他结果都没用了，可删

4、eval
首先将原来train的数据处理成anserini中qrel的格式：

调用指令
python BM25-evaluation/scripts/convert_train_to_trec_qrels.py --input /share/peitian/Data/Datasets/llm-embedder/qa/msmarco/train.json --output /share/yutao/yifei/bm25_data/qrels.tsv

之后调用eval函数：
python scripts/msmarco_passage_eval.py \
/share/yutao/yifei/bm25_data/qrels.tsv /share/yutao/yifei/bm25_data/output_runs/run.msmarco.tsv

会显示出MRR @10结果应该如下：

#####################
MRR @10: 0.23002808610498265
QueriesRanked: 400775
#####################

跋：
在scripts中有一些代码是之前处理数据用的，后来优化了上述指令中用到的代码，一些冗余的代码就保留未删
冗余代码也先别删，没准后面有用🤔